# [Serialization](https://docs.ray.io/en/latest/serialization.html)
Since Ray processes do not share memory space, data transferred between workers and nodes will need to serialized and deserialized. 

Ray uses the [**Plasma object store**](https://arrow.apache.org/docs/python/plasma.html) to efficiently transfer objects across different processes and different nodes. 

Numpy arrays in the object store are shared between workers on the same node (zero-copy deserialization).



In [2]:
import logging
import ray

In [2]:
import numpy as np
obj = [np.zeros(42)] * 99
l = ray.get(ray.put(obj))
assert l[0] is l[1]  # no problem!

2021-12-22 18:20:18,603	INFO services.py:1340 -- View the Ray dashboard at http://127.0.0.1:8265
2021-12-22 18:20:18,606	WARNING services.py:1826 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=4.97gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.


## the ways to serization, customized class

In [3]:
import time

# `la` in taiwanese means stir

@ray.remote
class Cup:
    def __init__(self, capacity):
        self.capacity = capacity
    
    def la_with_laable_item(self, laable_item):
        if laable_item.la():
            time.sleep(2)
            self.capacity -= 1
            
    def is_good(self):
        return self.capacity <= 0
        

class Leborn:
    def __init__(self, obj_id):
        self.obj_id = obj_id
        
    def la(self): 
        return False
    
    # without '__reduce__', the instance is unserializable.
    def __reduce__(self):
        deserializer = Leborn
        serialized_data = (self.obj_id, )
        return deserializer, serialized_data
    
    
class Spoon:
    def __init__(self, obj_id):
        self.obj_id = obj_id

    def la(self):
        return True
    
    # without '__reduce__', the instance is unserializable.
    def __reduce__(self):
        deserializer = Spoon
        serialized_data = (self.obj_id, )
        return deserializer, serialized_data

In [23]:
%%time

cup1 = Cup.remote(3)
cup2 = Cup.remote(3)

CPU times: user 4.44 ms, sys: 0 ns, total: 4.44 ms
Wall time: 2.43 ms


In [22]:
sp = Spoon("good good spoon")
lb = Leborn("good good Leborn")

### What if we `La` with Spoon?


In [19]:
%%time
 
print('la first time')
cup1.la_with_laable_item.remote(sp)
print('la first time sent')
print(f"is it good drink, {ray.get(cup1.is_good.remote())}")

print('la second time')
cup1.la_with_laable_item.remote(sp)
print('la second time sent')
print(f"is it good drink, {ray.get(cup1.is_good.remote())}")

print('la third time')
cup1.la_with_laable_item.remote(sp)
print('la third time sent')
print(f"is it good drink, {ray.get(cup1.is_good.remote())}")

la first time
la first time sent
is it good drink, False
la second time
la second time sent
is it good drink, False
la third time
la third time sent
is it good drink, True
CPU times: user 312 ms, sys: 96.7 ms, total: 409 ms
Wall time: 6.02 s


after spent 6 seconds, cup good good

### What if we `La` with Leborn?

In [24]:
%%time
 
print('la first time')
cup2.la_with_laable_item.remote(lb)
print('la first time sent')
print(f"is it good drink, {ray.get(cup2.is_good.remote())}")

print('la second time')
cup2.la_with_laable_item.remote(lb)
print('la second time sent')
print(f"is it good drink, {ray.get(cup2.is_good.remote())}")

print('la third time')
cup2.la_with_laable_item.remote(lb)
print('la third time sent')
print(f"is it good drink, {ray.get(cup2.is_good.remote())}")

la first time
la first time sent
is it good drink, False
la second time
la second time sent
is it good drink, False
la third time
la third time sent
is it good drink, False
CPU times: user 11.1 ms, sys: 0 ns, total: 11.1 ms
Wall time: 9.49 ms


cup is not good yet, of-course, Leborn is not Laable

what we observed? the fn call of actor is not parallel execution (prevent risk condition)

In [27]:
%%time

# another trila, call three la, will is queued?
print("start to send las")
cup2.la_with_laable_item.remote(sp)
cup2.la_with_laable_item.remote(sp)
cup2.la_with_laable_item.remote(sp)
cup2.la_with_laable_item.remote(sp)
cup2.la_with_laable_item.remote(sp)
cup2.la_with_laable_item.remote(sp)
print("las sent")


start to send las
las sent
CPU times: user 0 ns, sys: 6.09 ms, total: 6.09 ms
Wall time: 4.24 ms


In [28]:
%%time

ray.get(cup2.is_good.remote())


CPU times: user 699 ms, sys: 191 ms, total: 890 ms
Wall time: 11.4 s


True